In [1]:
import pandas as pd

In [5]:
train_data = pd.read_csv("Featured_data.csv")
test_data = pd.read_csv("Featured_data_2025.csv")


train_data.drop(columns=['Unnamed: 0'], inplace=True)
test_data.drop(columns=['Unnamed: 0'], inplace=True)

Missing values check

In [8]:
missing_train = train_data.isnull().sum()
missing_test = test_data.isnull().sum()




W/L%    1
PW      0
SRS     0
NRtg    1
ORtg    0
TS%     0
DRtg    0
PA/G    1
eFG%    0
3P%     1
FG%     1
dtype: int64

In [9]:
train_data.fillna(train_data.mean(), inplace=True)

In [10]:
test_data.fillna(test_data.mean(), inplace=True)

Ridge Regression is a type of linear regression that includes regularization to prevent overfitting. Regularization adds a penalty to the model's loss function, discouraging it from assigning large weights to features, especially when those features highly correlated.
Ridge regression  = (X^TX + lambda* I)^(-1) X^Ty

where X is the explanatory matrix, lambda is the ridge coefficient greater than 1



In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Features and target variable
X = train_data.drop(columns=["W/L%"])
y = train_data["W/L%"]

# Splitting the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features (Ridge Regression is sensitive to scale)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_data.drop(columns=["W/L%"]))

# Ridge Regression model
ridge = Ridge(alpha=1.0)  # Default alpha = 1.0
ridge.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = ridge.predict(X_train_scaled)
y_val_pred = ridge.predict(X_val_scaled)
y_test_pred = ridge.predict(X_test_scaled)

# Evaluate the model
train_mae = mean_absolute_error(y_train, y_train_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

# Results
train_mae, val_mae, val_rmse


(0.0291206061281818, 0.03219382637905683, 0.0428489070874633)

In [15]:
# Predict W/L% for the 2025 dataset
test_data["Predicted W/L%"] = y_test_pred

# Rank teams based on predicted W/L%
test_data["Predicted Rank"] = test_data["Predicted W/L%"].rank(ascending=False).astype(int)

# Sort the teams by their predicted rank for standings
predicted_standings = test_data.sort_values(by="Predicted Rank")

predicted_standings[["Predicted W/L%", "Predicted Rank"]].head()


,Predicted W/L%,Predicted Rank
1,0.728834,1
0,0.715641,2
2,0.661014,3
3,0.644440,4
4,0.630070,5


In [16]:
predicted_standings

,W/L%,PW,SRS,NRtg,ORtg,TS%,DRtg,PA/G,eFG%,3P%,FG%,Predicted W/L%,Predicted Rank
1,0.867000,27,9.77,11.800000,121.9,0.622,110.1,110.200000,0.595,0.406000,0.504000,0.728834,1
0,0.821000,26,11.27,12.000000,115.5,0.571,103.5,103.100000,0.539,0.349000,0.464000,0.715641,2
2,0.733000,26,9.36,10.800000,120.7,0.593,109.9,109.800000,0.562,0.365000,0.456000,0.661014,3
3,0.667000,25,7.22,8.600000,116.8,0.591,108.2,113.000000,0.558,0.370000,0.486000,0.644440,4
4,0.667000,24,5.98,8.200000,121.1,0.609,112.9,110.100000,0.577,0.396000,0.497000,0.630070,5
6,0.633000,22,4.79,5.700000,117.6,0.593,111.9,111.800000,0.559,0.377000,0.486000,0.577663,6
5,0.690000,22,6.09,6.400000,112.9,0.542,106.5,106.300000,0.506,0.324000,0.442000,0.563049,7
7,0.571000,18,2.76,2.900000,118.1,0.598,115.2,116.100000,0.566,0.376000,0.496000,0.513039,8
10,0.517000,18,1.87,1.800000,111.2,0.577,109.4,106.900000,0.545,0.372000,0.457000,0.504060,9
11,0.571000,17,1.25,1.300000,113.1,0.589,111.8,111.200000,0.562,0.391000,0.481000,0.503878,10
